In [ ]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [ ]:
#Testing
#Site: https://www.rebgv.org/
#Segment: /market-watch/
#Old Format:
#Page: /MLS-HPI-home-price-comparison.hpi.<area>.<sub-area>.<property type>.<YYYY-m-d>.html
#Example: https://www.rebgv.org/market-watch/MLS-HPI-home-price-comparison.hpi.greater_vancouver.detached.2020-2-1.html
#New Format (as of August 23rd, 2020):
#Page: /MLS-HPI-home-price-comparison.hpi.<area>.<sub-area>.<property type, nullable>.<YYYY-m-d>.html
#Example: https://www.rebgv.org/market-watch/MLS-HPI-home-price-comparison.hpi.greater_vancouver.all.detached.2020-7-1.html

linked = 'https://www.rebgv.org/market-watch/MLS-HPI-home-price-comparison.hpi.all.all.all.2020-7-1.html'

response = requests.get(linked)
# soup
soup = BeautifulSoup(response.text, 'html.parser')

# div main class: hpi-graphics
# div content class: table-wrapper
result = soup.find_all('div', class_='table-wrapper')
result

In [ ]:
home_dict = []

for year in range(2016, 2021):
    for month in range(1, 13):
        print(str(year) + "-" + str(month))
        try:
            url = 'https://www.rebgv.org/market-watch/MLS-HPI-home-price-comparison.hpi.all.all.all.' + str(year) + '-' + str(month) + '-1.html';
            print(url)
            r = requests.get(url)
            soup = BeautifulSoup(r.text, 'html.parser')

            home_table = soup.find('div', class_="table-wrapper")
            #print(home_table)
            for home in home_table.find_all('tbody'):
                rows = home.find_all('tr')
                for row in rows:
                    area = row.find('td').text;
                    benchmark = row.find_all('td')[1].text
                    priceIndex = row.find_all('td')[2].text
                    oneMonthChange = row.find_all('td')[3].text
                    sixMonthChange = row.find_all('td')[4].text
                    oneYearChange = row.find_all('td')[5].text
                    threeYearChange = row.find_all('td')[6].text
                    fiveYearChange = row.find_all('td')[7].text
                    propertyType = row.find_all('td')[8].text
                    year = year;
                    month = month;

                    home_obj = {
                        "Area": area,
                        "Benchmark": benchmark,
                        "Price Index": priceIndex,
                        "1 Month +/-": oneMonthChange,
                        "6 Month +/-": sixMonthChange,
                        "1 Year +/-": oneYearChange,
                        "3 Year +/-": threeYearChange,
                        "5 Year +/-": fiveYearChange,
                        "Property Type": propertyType,
                        "Report Month": month,
                        "Report Year": year
                    }
                    home_dict.append(home_obj)  
        except:
            continue

#print(home_dict)

In [ ]:
vancouver_home_data = pd.DataFrame(home_dict)
vancouver_home_data
#4565 rows as of August 23rd, 2020.  Data to up July 2020.

In [ ]:
# Export file as a CSV, without the Pandas index, but with the header
vancouver_home_data.to_csv("Vancouver Real Estate Board Data 201601 - 202007.csv", index=False, header=True)